In [2]:
def test(test_data, model_state_path):
    !pip install torch_geometric
    import pandas as pd
    import re
    import spacy
    import torch
    from torch_geometric.data import HeteroData
    from torch_geometric.nn import HeteroConv, GATConv
    import numpy as np
    import networkx as nx
    import matplotlib.pyplot as plt
    import random
    from sklearn.model_selection import train_test_split
    from transformers import BertTokenizer, BertModel
    import seaborn as sns
    import torch.nn.functional as F
    import nltk
    import matplotlib.pyplot as plt
    from nltk.corpus import reuters
    from nltk.corpus import stopwords
    from nltk import PorterStemmer, WordNetLemmatizer
    from transformers import BertTokenizer, BertModel
    from sklearn.model_selection import KFold
    from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, classification_report, roc_curve, auc, precision_recall_curve, confusion_matrix

    nltk.download('stopwords')
    nltk.download('wordnet')

    # Function for text preprocessing and lemmatization
    def preprocess_and_lemmatize(text) :
        text = text.lower()
        text = re.sub(r'\?+', '?', text)
        text = re.sub(r'\!+', '!', text)
        text = re.sub(r'\/+', '', text)
        text = re.sub(r'`', '', text)
        text = re.sub(r'-lrb-', '-', text)
        text = re.sub(r'-rrb-', '-', text)
        text = re.sub(r'\-+', '-', text)
        text = re.sub(r"\b(i['`’]ve|i['`’]m|i['`’]ll)\b", lambda m: m.group(0).replace("'", " "), text)
        text = re.sub(r" - ", " . ", text)

        doc = nlp(text)
        lemmatized_tokens = [token.lemma_ for token in doc]
        lemmatized_text = ' '.join(lemmatized_tokens)
        lemmatized_text = re.sub(r"'s\b", "s", lemmatized_text)
        return lemmatized_text

    # Function to extract metaphor-related sentences
    def extract_metaphor_sentences(df):
        metaphor_keywords = {
                          0: 'road',
                          1: 'candle',
                          2: 'light',
                          3: 'spice',
                          4: 'ride',
                          5: 'train',
                          6: 'boat'
                      }

        imp_text = []

        for _, row in df.iterrows():
            metaphor_keyword = metaphor_keywords.get(row['metaphorID'], '')
            selected_sentence = ""

            for sentence in row['lemmatized_text'].split('.'):
                if metaphor_keyword in sentence:
                    # Break condition: prioritize sentences without specific punctuation
                    if not any(punct in sentence for punct in [' - ', ' , ', ' ; ', ' ? ', ' ! ']):
                        selected_sentence = sentence
                        break

                    # Secondary prioritization based on punctuation
                    for punct in [' , ', ' ; ', ' ? ', ' ! ', ' - ']:
                        if punct in sentence:
                            for sub_sentence in sentence.split(punct):
                                if metaphor_keyword in sub_sentence:
                                    selected_sentence = sub_sentence
                                    break
                            if selected_sentence:
                                break

            if selected_sentence:
                imp_text.append(selected_sentence)

        return imp_text

    def clean_and_process_data(df):
      metaphor_keywords = {
                          0: 'road',
                          1: 'candle',
                          2: 'light',
                          3: 'spice',
                          4: 'ride',
                          5: 'train',
                          6: 'boat'
                      }
      df['lemmatized_text'] = df['text'].apply(preprocess_and_lemmatize)

      # Extract metaphor-related sentences
      imp_text = extract_metaphor_sentences(df)

      # Create cleaned DataFrame with important metaphor-related sentences
      cleaned_df = pd.DataFrame({'text': imp_text})
      cleaned_df[["metaphorID", "label"]] = df.iloc[:, :2].values


      # Map labels to target binary values
      cleaned_df["label"] = cleaned_df["label"].map({True: 1, False: 0})

      # Map metaphorID to metaphor keywords
      cleaned_df["metaphor"] = cleaned_df["metaphorID"].map(metaphor_keywords)
      cleaned_df = cleaned_df.drop("metaphorID", axis=1)

      return cleaned_df



    def create_bert_embeddings(text_series):
        embeddings = []
        for text in text_series:
            inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
            with torch.no_grad():
                outputs = bert_model(**inputs)
            embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
        return torch.tensor(np.array(embeddings), dtype=torch.float)

    # Create a heterogeneous graph
    def create_hetero_graph(texts, metas):
        G = nx.Graph()
        word_to_doc_edges = []
        unique_words = set()

        for doc_id, (text, meta) in enumerate(zip(texts, metas)):
            words = set(str(text).lower().split() + str(meta).lower().split())
            unique_words.update(words)
            for word in words:
                word_to_doc_edges.append(('word', word, 'doc', doc_id))

        return list(unique_words), word_to_doc_edges

    class HeteroGNN(torch.nn.Module):
        def __init__(self, hidden_channels, out_channels, num_layers, heads=4):
            super().__init__()
            self.convs = torch.nn.ModuleList()
            for _ in range(num_layers):
                conv = HeteroConv({
                    ('word', 'to', 'doc'): GATConv((-1, -1), hidden_channels // heads, heads=heads, concat=True, add_self_loops=False),
                    ('doc', 'to', 'word'): GATConv((-1, -1), hidden_channels // heads, heads=heads, concat=True, add_self_loops=False),
                })
                self.convs.append(conv)
            self.lin = torch.nn.Linear(hidden_channels, out_channels)

        def forward(self, x_dict, edge_index_dict):
            for conv in self.convs:
                x_dict = conv(x_dict, edge_index_dict)
                x_dict = {key: x.relu() for key, x in x_dict.items()}
            logits = self.lin(x_dict['doc'])
            return F.softmax(logits, dim=1)

    # Training and evaluation functions
    def train(model, data, optimizer, criterion):
        model.train()
        optimizer.zero_grad()
        out = model(data.x_dict, data.edge_index_dict)
        loss = criterion(out, data['doc'].y)
        loss.backward()
        optimizer.step()
        return loss.item()

    def evaluate(model, data):

        model.eval()

        with torch.no_grad():
            # We are not leaking any data both x_dict, edge_index_dict both do not have any y values
            out = model(data.x_dict, data.edge_index_dict)
            pred = out.argmax(dim=1)
            return pred

    def compute_metrics(train_true, train_pred, train_probs, test_true, test_pred, test_probs):
        def calculate_metrics(y_true, y_pred, y_probs):
            metrics = {
                "Precision": precision_score(y_true, y_pred, average="binary"),
                "Recall": recall_score(y_true, y_pred, average="binary"),
                "F1-Score": f1_score(y_true, y_pred, average="binary"),
                "Accuracy": accuracy_score(y_true, y_pred),
                "ROC-AUC": roc_auc_score(y_true, y_probs[:, 1]) if y_probs is not None else None
            }
            return metrics

        train_metrics = calculate_metrics(train_true, train_pred, train_probs)
        test_metrics = calculate_metrics(test_true, test_pred, test_probs)

        df = pd.DataFrame({
            'Train': train_metrics,
            'Test': test_metrics
        }).transpose()

        display(df)

        print("\nClassification Report (Train):")
        print(classification_report(train_true, train_pred))

        print("\nClassification Report (Test):")
        print(classification_report(test_true, test_pred))

        # Create subplots
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16))

        # ROC-AUC plot
        for true, probs, name in [(train_true, train_probs, 'Train'), (test_true, test_probs, 'Test')]:
            fpr, tpr, _ = roc_curve(true, probs[:, 1])
            roc_auc = auc(fpr, tpr)
            ax1.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.2f})')
        ax1.plot([0, 1], [0, 1], 'k--')
        ax1.set_xlim([0.0, 1.0])
        ax1.set_ylim([0.0, 1.05])
        ax1.set_xlabel('False Positive Rate')
        ax1.set_ylabel('True Positive Rate')
        ax1.set_title('Receiver Operating Characteristic (ROC) Curve')
        ax1.legend(loc="lower right")

        # Precision-Recall plot
        for true, probs, name in [(train_true, train_probs, 'Train'), (test_true, test_probs, 'Test')]:
            precision, recall, _ = precision_recall_curve(true, probs[:, 1])
            ax2.plot(recall, precision, label=f'{name}')
        ax2.set_xlabel('Recall')
        ax2.set_ylabel('Precision')
        ax2.set_title('Precision-Recall Curve')
        ax2.legend()

        # Confusion Matrix for Train data
        train_cm = confusion_matrix(train_true, train_pred)
        sns.heatmap(train_cm, annot=True, fmt='d', cmap='Blues', ax=ax3)
        ax3.set_xlabel('Predicted')
        ax3.set_ylabel('Actual')
        ax3.set_title('Confusion Matrix (Train)')

        # Confusion Matrix for Test data
        test_cm = confusion_matrix(test_true, test_pred)
        sns.heatmap(test_cm, annot=True, fmt='d', cmap='Blues', ax=ax4)
        ax4.set_xlabel('Predicted')
        ax4.set_ylabel('Actual')
        ax4.set_title('Confusion Matrix (Test)')

        plt.tight_layout()
        plt.show()

        return df


    def compute_metrics(y_true, y_pred):
        metrics = {
            "Precision": precision_score(y_true, y_pred, average="macro"),
            "Recall": recall_score(y_true, y_pred, average="macro"),
            "F1-Score": f1_score(y_true, y_pred, average="macro"),
            "Accuracy": accuracy_score(y_true, y_pred),
        }


        return metrics



    nlp = spacy.load("en_core_web_sm")

    cleaned_dummy = clean_and_process_data(test_data)
    dummy_text, dummy_meta = cleaned_dummy["text"], cleaned_dummy["metaphor"]

    # Initialize BERT tokenizer and model
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained('bert-base-uncased')

    dummy_words, dummy_edges = create_hetero_graph(dummy_text, dummy_meta)
    dummy_word_features = create_bert_embeddings(dummy_words)
    dummy_doc_features = create_bert_embeddings([text + " " + meta for text, meta in zip(dummy_text, dummy_meta)])

    dummy_data = HeteroData()
    dummy_data['word'].x = dummy_word_features
    dummy_data['doc'].x = dummy_doc_features
    dummy_data['word', 'to', 'doc'].edge_index = torch.tensor([[dummy_words.index(edge[1]) for edge in dummy_edges], [edge[3] for edge in dummy_edges]], dtype=torch.long)
    dummy_data['doc'].y = torch.tensor(cleaned_dummy["label"])

    # Add reverse edges
    dummy_data['doc', 'to', 'word'].edge_index = dummy_data['word', 'to', 'doc'].edge_index.flip([0])


    best_params =  {'hidden_channels': 32, 'num_layers': 2, 'heads': 8, 'lr': 0.001}

    final_model = HeteroGNN(
      hidden_channels=best_params['hidden_channels'],
      out_channels= 2,
      num_layers=best_params['num_layers'],
      heads=best_params['heads']
    )


    final_model.load_state_dict(torch.load(model_state_path))

    final_model.eval()

    predicted = evaluate(final_model, dummy_data)

    results = compute_metrics(dummy_data['doc'].y, predicted)
    results_df = pd.DataFrame(data = [results.values()], columns = results.keys())

    print("Results on test data:")
    print()
    print()

    display(results_df)
    print()
    print()
    print()
    print()
    print()
    print()
    print()

    print(results_df)
    print()
    print()
    print()
    print()
    print()
    print()
    print()
    test_data["predicted"] = predicted
    test_data["predicted"] = test_data["predicted"].map({1: True, 0: False})

    test_data.to_csv("predictions.csv")

    return test_data



In [3]:
import pandas as pd

# You would have to upload model state dict, we have provided the state dictionary
# Change the data in the below line
test_data = pd.read_csv('train-1.csv').sample(n=100, random_state=42)


model_state_path = 'GNN_optimal_state_dict.pth'

predictions = test(test_data, model_state_path)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.7 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Results on test data:




<ipython-input-2-be7a7a64b496>:299: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  final_model.load_state_dict(torch.load(model_state_path))


,Precision,Recall,F1-Score,Accuracy
0,0.845238,0.886667,0.862069,0.89









   Precision    Recall  F1-Score  Accuracy
0   0.845238  0.886667  0.862069      0.89







